# Importing Dependencies and Declaring Global Variables

### In this part we are going to install needed libraries and declare some variable needed for training and testing models.

In [ ]:
from keras.applications import VGG16,ResNet50
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import splitfolders
import shutil
import random

NUM_CLASS = 2
IMAGE_SIZE = 224
BATCH_SIZE = 100
EPOCHS = 2

# Data Collecting and Splitting

### We are going to collect data from the kaggle dataset, and copy it into splitfolders and let split folders split it into 3 part inside the <code>data</code> folder. Next part is initialize the image data generator for easier training with image.

In [ ]:
file_path='dataset'
splitted_folder='data'
if not os.path.exists(splitted_folder):
   os.makedirs(splitted_folder)
else:
  # Deleting an non-empty folder
  shutil.rmtree(splitted_folder, ignore_errors=True)
  print("Deleted '%s' directory successfully" % splitted_folder)

  os.makedirs(splitted_folder)

def train_test_valid(images_folder=file_path,splitted_folder=splitted_folder):
  input_folder = images_folder
  output_folder = splitted_folder
  splitfolders.ratio(input_folder,output_folder, seed = 1337,ratio=(.8, 0.1,0.1), group_prefix = None)
train_test_valid()

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'data/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'data/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = data_generator.flow_from_directory(
    'data/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
steps_per_epoch_test = len(test_generator)

# Model Training and Validation

### Finally we are going to train VGG16 and ResNet50 pre-trained models

In [ ]:
import tensorflow as tf
import os

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
os.environ["XLA_FLAGS"] = '--xla_gpu_cuda_data_dir="/home/aprilia/anaconda3/envs/deeplearning"'

In [ ]:
vgg16_model = Sequential()
vgg16_model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
vgg16_model.add(layers.Dense(NUM_CLASS, activation='softmax'))

# since pre-trained model already trained so we dont need to train it again
vgg16_model.layers[0].trainable = False

vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_model.summary()

In [ ]:
resnet50_model = Sequential()
resnet50_model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
resnet50_model.add(layers.Dense(NUM_CLASS, activation='softmax'))

# since pre-trained model already trained so we dont need to train it again
resnet50_model.layers[0].trainable = False

resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_model.summary()

In [ ]:
vgg16_history = vgg16_model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [ ]:
resnet50_history = resnet50_model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [ ]:
vgg16_model.save('classifier_vgg16_model.h5')
resnet50_model.save('classifier_resnet50_model.h5')

# Model Testing

### Final part is testing our model how it perform with data that it not has been trained on

In [ ]:
resnet = tf.keras.models.load_model('classifier_resnet50_model.h5')
vgg = tf.keras.models.load_model('classifier_vgg16_model.h5')

Print model accuracy

In [ ]:
resnet_scores = resnet.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("resnet ",resnet.metrics_names[1], resnet_scores[1]*100))

vgg_scores = vgg.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("vgg ",vgg.metrics_names[1], vgg_scores[1]*100))

Predict test dataset

In [ ]:
resnet_predict = resnet.predict_generator(test_generator,steps=steps_per_epoch_test,verbose=1)
vgg_predict = vgg.predict_generator(test_generator,steps=steps_per_epoch_test,verbose=1)

Print 10 random dataset prediction result

In [ ]:
result_vgg = vgg_predict.argmax(axis=-1)
for i in range(10):
    if result_vgg[random.randint(0,4000)] == 0:
        print("Negative") 
    else:
        print("Positive")


In [ ]:
result_resnet = resnet_predict.argmax(axis=-1)
for i in range(10):
    if result_resnet[random.randint(0,4000)] == 0:
        print("Negative") 
    else:
        print("Positive")
